In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
df = pd.read_excel('../Especies_Sin_Foto_iNaturalist.xlsx')
df = df[df['taxonomicgroup'] == 'Plantas vasculares'][['taxonid', 'name']]
df['name'] = df['name'].apply(lambda x: '+'.join(x.split()[:2])) # Solo nombre científico simplificado
df['photo_link'] = ''
df['by'] = ''

In [ ]:
base_url = 'https://identify.plantnet.org/es/k-world-flora/species?search={species}&sortBy=name&sortOrder=asc'
driver = webdriver.Chrome()
for i, row in df.iterrows():
    print(f'Starting species {i+1}...', end='\r')
    driver.get(base_url.format(species=row['name']))
    try:
        plant = driver.find_element(By.CLASS_NAME, 'pn-species-name.fw-bold.m-0.fs-3')
        url = plant.find_element(By.TAG_NAME, 'a').get_attribute('href')
        driver.get(f"{url}#galleries")
        photo_item = driver.find_element(By.CLASS_NAME, 'position-relative')
        photo_link = photo_item.find_element(By.TAG_NAME, 'a').get_attribute('href')
        photo_item.click()
        username = driver.find_element(By.CLASS_NAME, 'text-truncate.user-link-name').text
        df.loc[i, 'photo_link'] = photo_link
        df.loc[i, 'by'] = username
    except:
        continue
    if (i+1) % 10:
        df.to_excel('plantnet_images.xlsx', index=False)
driver.close()
df = df[df['by'] != '']
df.to_excel('plantnet_images.xlsx', index=False)

In [ ]:
df = df[df['by'] != '']
df.to_excel('plantnet_images.xlsx', index=False)

In [ ]:
df_all = pd.read_excel('../Especies_Sin_Foto_iNaturalist.xlsx')
df_all = df_all[~df_all['taxonid'].isin(df['taxonid'])]
df_all.to_excel('../Especies_Sin_Foto_iNaturalist_Plantnet.xlsx', index=False)